# About
Implement clustering algorithms
1. K-Means
1. Heirarchical clustering
1. DBSCAN

## The Team
| Name| Student ID|
|------------|---------------|
|Cynthia Cai | 5625483 |
|Pratyush Kumar | 5359252|


# Imports

// add the imports to the cell below

In [3]:
import numpy as np 
import pandas as pd
from scipy.spatial import ConvexHull, distance_matrix
import glob
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="darkgrid")

# Reading the dataset


From the readme for the xyz files, we know that:

Ground truth labels:
|File range|Label|
|--|--|
|    000 - 099: |building|
|    100 - 199: |car|
|    200 - 299: |fence|
|    300 - 399: |pole|
|    400 - 499: |tree|


workflow:

iterate through the files, and collect them in a dataframe

Use [this link](https://pandas.pydata.org/docs/reference/api/pandas.concat.html#pandas.concat) for concatenating the dataframes

In [ ]:
xyzPath = './scene_objects/data/*.xyz'

dataPathsList = glob.glob(xyzPath)

In [ ]:
allPointsDF= pd.DataFrame(columns=['x','y','z', 'fileNo', 'groundLabel'])
# featureDF = pd.DataFrame(columns=['Label' , 'convHull', median] )

def df_maker(df1, df2):
    return pd.concat([df1, df2], sort=False, ignore_index=True)

labelToGive = None
for path in dataPathsList:
    indx = int(path.split('/')[-1][0:3])
    # if else to determine label
    if indx>=0 and indx<100:
        labelToGive = 'building' 
    elif indx>=100 and indx<200:
        labelToGive = 'car' 
    elif indx>=200 and indx<300:
        labelToGive = 'fence' 
    elif indx>=300 and indx<400:
        labelToGive = 'pole' 
    elif indx>=400 and indx<500:
        labelToGive = 'tree' 

    # print(indx, labelToGive)        

    # using pandas to read dataset and make a dataFrame
    tempDF = pd.read_csv(path, delimiter=' ', header=None, dtype=np.float64, names=['x','y','z'])
    tempDF.loc[:,'fileNo'] = indx
    tempDF.loc[:,'groundLabel'] = labelToGive

    # merge with megaDFofPoints
    allPointsDF = df_maker(allPointsDF, tempDF)

# allPointsDF.head()

In [ ]:
# save to pickle file
# allPointsDF.to_pickle('./scene_objects/compressedData.pkl')

## Making feature points
Identified feature points: `//add more`
* median height(z)
* convex hull

In [ ]:
def label_determiner(indx):
    labelToGive=None
    if indx>=0 and indx<100:
        labelToGive = 'building' 
    elif indx>=100 and indx<200:
        labelToGive = 'car' 
    elif indx>=200 and indx<300:
        labelToGive = 'fence' 
    elif indx>=300 and indx<400:
        labelToGive = 'pole' 
    elif indx>=400 and indx<500:
        labelToGive = 'tree' 
    return labelToGive


featureDF = allPointsDF.groupby('fileNo').var()
featureDF.rename(columns={'x':'varX','y':'varY','z':'varZ'}, inplace=True)
featureDF.loc[:,'median_Z'] = allPointsDF.groupby('fileNo').z.median()
# featureDF.loc[:,'mean_Z'] = allPointsDF.groupby('fileNo').z.mean()

# range of x,y,z
featureDF.loc[:,'range_X'] = allPointsDF.groupby('fileNo').x.max() - allPointsDF.groupby('fileNo').x.min()
featureDF.loc[:,'range_Y'] = allPointsDF.groupby('fileNo').y.max() - allPointsDF.groupby('fileNo').y.min()
featureDF.loc[:,'range_Z'] = allPointsDF.groupby('fileNo').z.max() - allPointsDF.groupby('fileNo').z.min()

featureDF.loc[:,'Volume'] = allPointsDF.set_index('fileNo').loc[:,'x':'z'].groupby('fileNo').apply(ConvexHull).apply(lambda x: x.volume)

# points density
featureDF.loc[:,'footprintDensity'] =  allPointsDF.groupby('fileNo').count().x / (featureDF.range_X * featureDF.range_Y)
featureDF.loc[:,'volumeDensity'] =  allPointsDF.groupby('fileNo').count().x / featureDF.Volume

featureDF.loc[:,'label'] = featureDF.reset_index().fileNo.apply(label_determiner)

# standardize DF
standardFeatureDF = (featureDF.iloc[:,:-1] - featureDF.iloc[:,:-1].mean() ) / featureDF.iloc[:,:-1].std()

# join labels to the feature DF
standardFeatureDF = standardFeatureDF.join(other=featureDF.label ,on='fileNo')

featureDF.to_pickle('./scene_objects/featureData.pkl')
standardFeatureDF.to_pickle('./scene_objects/standardFeatureData.pkl')

### Plotting to see resemblamces and clusters, if any
needed: seaborn

In [2]:
# load df's
featureDF = pd.read_pickle('./scene_objects/featureData.pkl')
standardFeatureDF = pd.read_pickle('./scene_objects/standardFeatureData.pkl')

In [ ]:
sns.pairplot(data=featureDF, hue="label")

normalize the feature df </br>
[from stackoverflow we see](https://stackoverflow.com/questions/26414913/normalize-columns-of-pandas-data-frame), that we can just use pandas for a standard scaling, or else, a [standard scaler from sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) can also be applied </br>

from [answer here](https://stats.stackexchange.com/questions/417339/data-standardization-vs-normalization-for-clustering-analysis), we see that standard scaler is used for k means , so we are going with that

In [ ]:
sns.pairplot(data=standardFeatureDF, hue="label")

# Clustering Algorithms
note: already loaded the featureDF and standardised in the cell above

## K-Means clustering

In [ ]:


def k_means():
    """
    summary: this function is not yet ready
    """
    pass

In [ ]:
k_means?

## Heirarchical clustering

This [ref was nice](https://www.section.io/engineering-education/hierarchical-clustering-in-python/) for heirarchical clustering understanding
Some other sources:
* [Statquest](https://www.youtube.com/watch?v=7xHsRkOdVwo&ab_channel=StatQuestwithJoshStarmer)
* Penn state [pseudo code](https://online.stat.psu.edu/stat508/lesson/12/12.7)
* pseudo code from [researchgate](https://www.researchgate.net/figure/The-hierarchical-clustering-algorithm-in-pseudocode_fig1_202144697)
* towards data science article to do [step by step](https://towardsdatascience.com/breaking-down-the-agglomerative-clustering-process-1c367f74c7c2) {this is a good one to follow}
* another one [for theory](https://towardsdatascience.com/machine-learning-algorithms-part-12-hierarchical-agglomerative-clustering-example-in-python-1e18e0075019)
* similar [theory as above](https://www.geeksforgeeks.org/ml-hierarchical-clustering-agglomerative-and-divisive-clustering/)
* real good [step by step explaination](https://medium.com/@darkprogrammerpb/agglomerative-hierarchial-clustering-from-scratch-ec50e14c3826), also the [github code](https://github.com/Darkprogrammerpb/DeepLearningProjects/blob/master/Project40/agglomerative_hierarchial_clustering/Hierarchial%20Agglomerative%20clustering.ipynb)

### To Think in heirarchical clustering:
* Which type of heirarchical clustering are we doing: lets begin with agglomerative clustering
* Within the selected type what distance metrics are we using



devise new distance matrix and then repeat the sequence:
### TODO: 
* linkage between the clusters
* updation of the distance matrix

clusters to be made:
`vals.idxmin()` and `idVals.iloc[vals.idxmin()]`

* inter cluster distance 
    * threshold
* distance types
    * calculate distance yourself
* convert to function
* check for number of clusters

In [6]:
def heirarchicalClusterer(standardizedDF = standardFeatureDF, linkageType='complete'):
    """
    Summary:
            This function takes in a standardized dataframe with features of objects and returns a list of clusters based on the indices of the dataframe, 
            note: the distances are calculated on basis of minkowski distance
    Arguments:
            standardizedDF (pd.DataFrame): standardized dataframe
            linkageType (str) : can be 'complete' for complete linkage, 'single' for single linkage and 'average' for average linkage
    Return (list) : a list of indices based on the dataframe input, with clusters inside the list, every list will have a length of 2
    """        
    tempDF = standardizedDF.iloc[:,:-1].copy()

    distMatDF = pd.DataFrame( distance_matrix(tempDF.values, tempDF.values), index = tempDF.index, columns = tempDF.index)
    # replace 0 distances with np.nan
    distMatDF = distMatDF.where(distMatDF!=0, np.nan)
        
    # clustCHECK WILL have two nodes each and a full node
    clustCheck = {}
    iterationCounter=0
    m=len(distMatDF)

    while m>1: 

        # cluster size
        # print(f"Total sample = {m}")
        # compute distances

        # get indices with min dist
        vals = distMatDF.min(skipna=True)
        idVals = distMatDF.idxmin(skipna=True)

        ind_to_pop = [idVals.loc[vals.idxmin()] , vals.idxmin()]
        # update distmatrix 
        ## add updated new row, col to dist mat  
        ## this updated row is basically the minimum of the two eliminated rows
        if linkageType=='complete':
            singleLink_minRow = distMatDF.loc[ind_to_pop].drop(ind_to_pop, axis=1).max()
        elif linkageType=='single':
            singleLink_minRow = distMatDF.loc[ind_to_pop].drop(ind_to_pop, axis=1).min()
        elif linkageType=='average':
            singleLink_minRow = distMatDF.loc[ind_to_pop].drop(ind_to_pop, axis=1).mean()

        singleLink_minRow.rename(f"cluster {iterationCounter}", inplace=True)

        # pop row and col from dist mat
        distMatDF = distMatDF.drop(ind_to_pop, axis=0).drop(ind_to_pop, axis=1)

        # min distance from other points
        distMatDF = distMatDF.append(singleLink_minRow)
        distMatDF.loc[:,singleLink_minRow.name] = singleLink_minRow
        # update value of m
        m = len(distMatDF)

        indPop1, indPop2 = ind_to_pop

        clustCheck[f"cluster {iterationCounter}"] = {'node1':indPop1 , "node2":indPop2, 'fullnodes':ind_to_pop}
        # print("before" , clustCheck[f'cluster {iterationCounter}'])
        
        # Case: if first index is a cluster
        if (indPop1 in clustCheck.keys()) and (indPop2 in clustCheck.keys()): #both are clusters
            clustCheck[f"cluster {iterationCounter}"] = {'node1':clustCheck[indPop1]['fullnodes'].copy() , "node2":clustCheck[indPop2]['fullnodes'].copy() }
            tempFull = [clustCheck[f"cluster {iterationCounter}"]["node1"].copy(), clustCheck[f"cluster {iterationCounter}"]["node2"].copy()]
            clustCheck[f"cluster {iterationCounter}"]["fullnodes"] = tempFull  


        # Case: if first index is a cluster
        elif indPop1 in clustCheck.keys(): #means first position is cluster
            clustCheck[f"cluster {iterationCounter}"] = {'node1':clustCheck[indPop1]['fullnodes'].copy() , "node2":indPop2 }
            tempFull = [clustCheck[f"cluster {iterationCounter}"]["node1"].copy() , clustCheck[f"cluster {iterationCounter}"]["node2"]]
            clustCheck[f"cluster {iterationCounter}"]["fullnodes"] = tempFull

        # Case: if second index is a cluster
        elif indPop2 in clustCheck.keys(): #means first position is cluster
            clustCheck[f"cluster {iterationCounter}"] = {'node1':indPop1 , "node2":clustCheck[indPop2]['fullnodes'].copy()}
            tempfull=[clustCheck[f"cluster {iterationCounter}"]["node1"].copy() , clustCheck[f"cluster {iterationCounter}"]["node2"]]
            clustCheck[f"cluster {iterationCounter}"]["fullnodes"] =  tempFull

        iterationCounter+=1
    return clustCheck



In [7]:
clustCheck = heirarchicalClusterer()
clustersList = clustCheck[list(clustCheck.keys())[-1]]["fullnodes"]


def flatten(l):
    try:
        return flatten(l[0]) + (flatten(l[1:]) if len(l) > 1 else []) if type(l) is list else [l]
    except IndexError:
        return []

# clusters are hardcoded in the following variables
cluster1Indices = flatten(clustersList[0][0][0][0])
cluster2Indices = flatten(clustersList[0][0][0][1])
cluster3Indices = flatten(clustersList[0][0][1])
cluster4Indices = flatten(clustersList[0][1])
cluster5Indices = flatten(clustersList[1])


HeirarchicalClusters = [standardFeatureDF.loc[i] for i in [cluster1Indices,cluster2Indices,cluster3Indices,cluster4Indices,cluster5Indices] ]


In [9]:
HeirarchicalClusters[0]

,varX,varY,varZ,median_Z,range_X,range_Y,range_Z,Volume,footprintDensity,volumeDensity,label
fileNo,,,,,,,,,,,
398,-0.134515,-0.181069,2.372077,2.388115,-0.123250,-0.151116,2.006573,-0.187196,-1.265365,-0.944853,pole
376,-0.126178,-0.167149,3.100400,1.724036,-0.171041,-0.184493,1.743910,-0.206562,-1.394470,-0.995891,pole
373,-0.178737,-0.226653,3.609181,1.950260,-0.598798,-0.581903,2.140092,-0.274912,-0.578723,-0.814868,pole
372,-0.177238,-0.228530,3.747861,2.259191,-0.431926,-0.604852,2.212325,-0.263715,-1.013897,-0.907648,pole
384,-0.175347,-0.222409,3.739850,2.752994,-0.481281,-0.501589,2.562541,-0.250250,-0.747503,-0.859952,pole
...,...,...,...,...,...,...,...,...,...,...,...
257,0.403866,1.783089,-0.549377,-0.667137,1.541562,2.551481,-0.709792,0.285922,-1.487444,-0.984102,fence
297,-0.181122,0.419389,-0.553001,-0.903092,-0.767238,1.088053,-0.749191,-0.290000,-0.333193,0.049062,fence
258,-0.181152,0.562643,-0.511910,-0.620919,-0.770370,1.490679,-0.836745,-0.286688,0.204018,0.444854,fence


In [10]:
HeirarchicalClusters[1]

,varX,varY,varZ,median_Z,range_X,range_Y,range_Z,Volume,footprintDensity,volumeDensity,label
fileNo,,,,,,,,,,,
295,2.266812,-0.232242,-0.540371,-1.202293,3.870733,-0.615285,-0.718547,-0.168536,-1.317988,-0.829887,fence
265,1.439334,-0.095107,-0.569623,-1.357974,3.035584,0.153461,-0.895844,0.035247,-1.413797,-0.911488,fence
288,3.627950,0.369046,-0.524921,-0.800926,4.728600,0.977487,-0.458074,1.187799,-1.445213,-0.999585,fence
213,6.161008,0.527408,-0.541742,-0.589296,5.834819,2.335567,-0.504040,3.242568,-1.492931,-1.035882,fence
226,0.944699,0.755594,-0.548783,-0.890930,2.112691,2.092530,-0.731680,0.580761,-1.467470,-0.990374,fence
224,0.706256,1.125417,-0.539418,-0.759573,2.218455,2.133208,-0.563139,0.730994,-1.476915,-1.006320,fence
261,1.286509,0.566682,-0.552818,-1.036881,2.932172,1.976749,-0.720736,0.647553,-1.475880,-0.987651,fence
286,2.035528,0.705688,-0.539929,-1.063639,2.833456,1.162111,-0.377086,1.332857,-1.436936,-1.020340,fence
252,1.782786,1.263814,-0.548217,-1.151210,3.919306,2.488896,-0.606916,1.452026,-1.496967,-1.021547,fence


In [11]:
HeirarchicalClusters[2]

,varX,varY,varZ,median_Z,range_X,range_Y,range_Z,Volume,footprintDensity,volumeDensity,label
fileNo,,,,,,,,,,,
387,-0.173474,-0.225929,6.960877,3.520455,-0.464829,-0.440047,3.234518,-0.218391,-0.985257,-0.958692,pole
377,-0.177273,-0.227282,6.846214,3.295447,-0.518102,-0.625714,3.140398,-0.263982,-0.645465,-0.866373,pole
444,-0.117950,-0.135400,7.418060,3.638433,0.179943,0.225433,3.794864,0.808460,1.918385,-0.736611,tree


In [12]:
HeirarchicalClusters[3]

,varX,varY,varZ,median_Z,range_X,range_Y,range_Z,Volume,footprintDensity,volumeDensity,label
fileNo,,,,,,,,,,,
247,20.090193,0.405142,-0.518956,-0.934715,12.074141,1.227824,-0.563139,4.346387,-1.471782,-1.025261,fence


In [13]:
HeirarchicalClusters[4]

,varX,varY,varZ,median_Z,range_X,range_Y,range_Z,Volume,footprintDensity,volumeDensity,label
fileNo,,,,,,,,,,,
60,0.383449,20.204525,-0.436967,0.111271,1.662997,12.060119,0.776438,18.569752,0.410618,-0.766191,building


## DBSCAN

In [8]:
HeirarchicalClusters[0]

,varX,varY,varZ,median_Z,range_X,range_Y,range_Z,Volume,footprintDensity,volumeDensity,label
fileNo,,,,,,,,,,,
398,-0.134515,-0.181069,2.372077,2.388115,-0.123250,-0.151116,2.006573,-0.187196,-1.265365,-0.944853,pole
376,-0.126178,-0.167149,3.100400,1.724036,-0.171041,-0.184493,1.743910,-0.206562,-1.394470,-0.995891,pole
373,-0.178737,-0.226653,3.609181,1.950260,-0.598798,-0.581903,2.140092,-0.274912,-0.578723,-0.814868,pole
372,-0.177238,-0.228530,3.747861,2.259191,-0.431926,-0.604852,2.212325,-0.263715,-1.013897,-0.907648,pole
384,-0.175347,-0.222409,3.739850,2.752994,-0.481281,-0.501589,2.562541,-0.250250,-0.747503,-0.859952,pole
...,...,...,...,...,...,...,...,...,...,...,...
257,0.403866,1.783089,-0.549377,-0.667137,1.541562,2.551481,-0.709792,0.285922,-1.487444,-0.984102,fence
297,-0.181122,0.419389,-0.553001,-0.903092,-0.767238,1.088053,-0.749191,-0.290000,-0.333193,0.049062,fence
258,-0.181152,0.562643,-0.511910,-0.620919,-0.770370,1.490679,-0.836745,-0.286688,0.204018,0.444854,fence


# Validation
* take numbner of output classification in a cluster and check with the actual calues

In [ ]:
def validateModels(classifiedData, originalData):
    pass